In [1]:
import pandas 
import selenium
import numpy 
import seaborn
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def scrapeComments(route_id):
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    route_com = driver.find_element_by_xpath("//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")
    comment_body = route_com.find_elements_by_class_name('comment-body')
    func_comments = []
    for i in comment_body: 
        func_comments.append(i.text)
    return(func_comments)


In [89]:
url_split = driver.current_url.split('/')
route_id = url_split[4]
route_name = url_split[5]
scrapeComments(route_id)


['Yet another great Devore route. 7 clean pitches with distinct, well protected cruxes, followed by fun moderate climbing. Mostly enjoyable slab and thin cracks. Pitch 3 is not to be missed.\n\nGear suggestion: single rack of cams to 2" plus doublees of small nuts and cams. 70m rope Feb 27, 2012',
 'Oh what a great ride.\n\nI definitely give it a solid 10d as the crux moves on P3 are just as sustained as the traverse moves over on Hall Pass on 3 Hour Arete.\n\n\nDefintitely the PSOM Wall Classic for sure. Sep 23, 2012',
 "What a great route. Rick Ziegler's recommendation for the rack is spot on, tiny nuts and cams for the crux pitch. I climbed this two days ago and my mind is still blown. Incredible route, well protected cruxes, beautiful movement, awesome position...definitely a 'highlight reel' route. Go do it. Oct 8, 2012",
 'Did the first 5 pitches of this one day before yesterday. Yes to all the props above. Super fun well protected climbing. Also did 3 hour arete and silver strea

In [3]:
#Read in Config txt file as dict
with open(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\config.txt", "r") as config_txt:
    config_txt = eval(config_txt.read())
config = config_txt

#location 
state = str(config_txt['state_location'])
sub_location = str(config_txt['sub_location'])


#mapping rock route type to HTML id + map other settings to vars 
route_type = config["route_type"].lower()
diff_rating_range = config["diff_rating_range"]

rock_type_dict = {"sport":"check_is_sport_climb", "trad":"check_is_trad_climb", "toprope" : "check_is_top_rope"}
rock_type = config["rock_route_type_exclude"]

In [3]:
location
print("California", location)

NameError: name 'location' is not defined

In [7]:
driver = webdriver.Chrome(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\chromedriver.exe")
driver.get("http://www.mountainproject.com")
wait = WebDriverWait(driver, 10)

# Apply Search Settings 

In [8]:
#apply config settings to "route finder"
select_type = Select(driver.find_element_by_id("type")) 
select_type.select_by_value(route_type)

#lb = lower bound, ub = upper bound
if route_type =='rock': 
    Select(driver.find_element_by_id("diffMinrock")).select_by_visible_text(diff_rating_range[0])
    Select(driver.find_element_by_id("diffMaxrock")).select_by_visible_text(diff_rating_range[1])
    for rock_toggle in rock_type: 
        if rock_type[rock_toggle] == True:
            driver.find_element_by_id(rock_type_dict[rock_toggle]).click()
            
if state != '': 
    driver.find_element_by_xpath('//*[@id="routeFinderForm"]/table/tbody/tr[4]/td[2]/a').click()
    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, state))).click()
    if sub_location != '': 
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, sub_location))).click()
    driver.find_element_by_id("select-area").click()
        
        
driver.find_element_by_xpath("//*[@id='routeFinderForm']/table/tbody/tr[5]/td/input").click()
wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "View All"))).click()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="https://www.mountainproject.com/area/105708959/california">...</a> is not clickable at point (250, 825). Other element would receive the click: <div class="modal fade in" id="global-modal" tabindex="-1" role="dialog" data-orig-full-path="/" style="display: block; padding-right: 17px;">...</div>
  (Session info: chrome=85.0.4183.102)


In [54]:
driver.find_element_by_xpath("//*[@id='route-page']/div/div[1]/h1").text

'John Fischer Memorial Route'

In [9]:
num_routes = (driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[2]/div/div[1]").text).split('Results 1 to ')[1].split(' ')[0]
comm_dict = {}
for route in range(2, int(num_routes)+2): 
    driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[3]/div/div/table[2]/tbody/tr["+str(route)+"]/td[1]/a").click() 
    url_split = driver.current_url.split('/')
    route_id = url_split[4]
    route_name = url_split[5]
    comm_dict[route_name] = scrapeComments(route_id)
    driver.back()


KeyboardInterrupt: 

In [10]:
comm_dict

{'john-fischer-memorial-route': [],
 'cromagnon': [],
 'superfly': [],
 'gorgeous': [],
 'wacked-scenario': [],
 'sendero-luminoso': [],
 'lava-haul': [],
 'he-she': [],
 'hardly-wallbanger': [],
 'dr-evil': [],
 'borrowing-from-tradition': [],
 'yellow-peril': [],
 'light-within': [],
 'black-lassie': ['Good holds and good stances make this a terrific warm up. Less pumpy than that .10a! Aug 21, 2008',
  'The most fun I\'ve had on a climb in a while! My follower thought it was easier than the 10A "Cromagnon". The crux is in the 2nd half and involves trying to prevent a barn door, but it\'s very well protected. Aug 15, 2010',
  "one of the best 10s I've ever done. Really good movement and balancy at certain sections Oct 7, 2015"],
 'machine-gun-jumblies': []}

In [ ]:
//*[@id="comments-Climb-Route-105912080"]

In [48]:
route_table = driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[3]/div/div/table[1]")
table_rows = route_table.find_elements_by_xpath("//tbody")


table_rows[0].click()

In [10]:
#comments scrape 

test = driver.find_element_by_xpath("//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")
test2 = test.find_elements_by_class_name('comment-body')
for i in test2: 
    print(i.text, end = '\t NEWB')

NameError: name 'route_id' is not defined

In [71]:
"//*[@id='comments-Climb-Route-" + str(105843581) + "']/div[2]/div[2]"

"//*[@id='comments-Climb-Route-105843581']/div[2]/div[2]"

In [55]:
//*[@id="comments-Climb-Route-105912080"]/div[2]/div[2]

In [ ]:
//*[@id="comments-Climb-Route-105843581"]/div[2]/div[2]